In [6]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.nn import GATConv
from sklearn.metrics import mean_absolute_error, r2_score, accuracy_score, confusion_matrix, f1_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# Enable detailed CUDA error reporting
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ['TORCH_USE_CUDA_DSA'] = "1"

# Define MTAD-GAT model with GRU layer
class MTADGAT(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_heads):
        super(MTADGAT, self).__init__()
        self.gat1 = GATConv(input_dim, hidden_dim, heads=num_heads, concat=True)
        self.gat2 = GATConv(hidden_dim * num_heads, output_dim, heads=1, concat=True)
        self.gru = nn.GRU(output_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x, edge_index):
        x = self.gat1(x, edge_index)
        x = torch.relu(x)
        x = self.gat2(x, edge_index)
        x = x.view(1, -1, x.size(1))  # Reshape for GRU
        x, _ = self.gru(x)
        x = self.fc(x.view(-1, x.size(2)))
        return x

# Load data function
def load_data(files):
    df_list = []
    for file in files:
        df = pd.read_csv(file)
        df.drop(columns=['time'], inplace=True)  # 'time' column removal
        df_list.append(df)
    combined_df = pd.concat(df_list, axis=0, ignore_index=True)
    return combined_df

# File paths
train_files = [
    'C:/Users/Researcher/Desktop/MTAD-GAT/train/train1.csv',
    'C:/Users/Researcher/Desktop/MTAD-GAT/train/train2.csv'
]
test_files = [
    'C:/Users/Researcher/Desktop/MTAD-GAT/test1/test1.csv',
    'C:/Users/Researcher/Desktop/MTAD-GAT/test1/test2.csv'
]

# Load data
train_df = load_data(train_files)
test_df = load_data(test_files)

# Data preparation function
def prepare_data(df, label_column='attack'):
    inputs = df.drop(columns=[label_column]).values.astype(float)
    labels = df[label_column].values.astype(float)
    return torch.tensor(inputs, dtype=torch.float32), torch.tensor(labels, dtype=torch.float32)

# Data preprocessing
scaler = StandardScaler()
train_scaled = scaler.fit_transform(train_df.drop(columns=['attack']))
test_scaled = scaler.transform(test_df.drop(columns=['attack']))

train_scaled_df = pd.DataFrame(train_scaled, columns=train_df.columns.drop('attack'))
test_scaled_df = pd.DataFrame(test_scaled, columns=test_df.columns.drop('attack'))

train_scaled_df['attack'] = train_df['attack'].values
test_scaled_df['attack'] = test_df['attack'].values

train_inputs, train_labels = prepare_data(train_scaled_df, label_column='attack')
test_inputs, test_labels = prepare_data(test_scaled_df, label_column='attack')

train_dataset = torch.utils.data.TensorDataset(train_inputs, train_labels)
test_dataset = torch.utils.data.TensorDataset(test_inputs, test_labels)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

# Function to create edge_index based on correlation matrix
def create_edge_index(corr_matrix, threshold=0.5):
    edge_index = []
    num_nodes = corr_matrix.shape[0]
    for i in range(num_nodes):
        for j in range(num_nodes):
            if i != j and abs(corr_matrix[i, j]) >= threshold:
                edge_index.append([i, j])
    return torch.tensor(edge_index, dtype=torch.long).t().contiguous()

# Compute correlation matrix
corr_matrix = train_scaled_df.drop(columns=['attack']).corr().values

# Create edge_index
num_nodes = train_scaled_df.drop(columns=['attack']).shape[1]
edge_index = create_edge_index(corr_matrix, threshold=0.5)

# Model initialization and training setup
input_dim = num_nodes  # Input dimension
hidden_dim = 64
output_dim = 32
num_heads = 8

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MTADGAT(input_dim, hidden_dim, output_dim, num_heads)

# Move model to GPU
model = model.to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Model training
epochs = 50
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Check the shapes and values
        assert inputs.size(0) == labels.size(0), f"Batch size mismatch: {inputs.size(0)} vs {labels.size(0)}"
        assert not torch.any(torch.isnan(inputs)), "NaNs found in inputs"
        assert not torch.any(torch.isnan(labels)), "NaNs found in labels"

        optimizer.zero_grad()
        outputs = model(inputs, edge_index.to(device))
        loss = criterion(outputs.squeeze(), labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_loader)}")

    # Save model checkpoint
    torch.save(model.state_dict(), f'model_checkpoint_epoch_{epoch + 1}.pth')

# Model evaluation
model.eval()
predictions = []
with torch.no_grad():
    for inputs, _ in test_loader:
        inputs = inputs.to(device)
        outputs = model(inputs, edge_index.to(device))
        predictions.append(outputs.cpu().numpy())
predictions = np.concatenate(predictions)

# Evaluation metrics
mse = np.mean((test_labels.numpy() - predictions.flatten()) ** 2)
mae = mean_absolute_error(test_labels.numpy(), predictions)
r2 = r2_score(test_labels.numpy(), predictions)

print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")
print(f"R^2 Score: {r2}")

# Anomaly detection
anomaly_threshold = mse + 3 * np.std(predictions - test_labels.numpy().reshape(-1, 1))

anomalies = (np.abs(predictions - test_labels.numpy().reshape(-1, 1)) > anomaly_threshold).astype(int)

# Classification metrics
accuracy = accuracy_score(test_labels.numpy(), anomalies)
f1 = f1_score(test_labels.numpy(), anomalies, average='macro')
cm = confusion_matrix(test_labels.numpy(), anomalies)

print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")
print("Confusion Matrix:")
print(cm)

sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Normal', 'Anomaly'], yticklabels=['Normal', 'Anomaly'])
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix')
plt.show()

# Visualization of predictions and anomalies
plt.figure(figsize=(15, 5))
plt.plot(test_labels.numpy(), label='Actual', color='blue')
plt.plot(predictions, label='Predicted', color='green')
plt.scatter(np.where(anomalies == 1)[0], predictions[anomalies == 1], color='red', label='Anomalies')
plt.legend()
plt.show()


RuntimeError: CUDA error: device-side assert triggered
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
